# Demand side flexibility


In [192]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [193]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [194]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [195]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [196]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [197]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame)

Row,CO2 Intensity
,Int64
1,0
2,0
3,452
4,965
5,0
6,0


In [198]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [199]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [200]:
#demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
demand = Matrix(CSV.read("demand_flex_vre50.csv", DataFrame)) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 1)


5-element Vector{Float64}:
 608.397763758
 627.358644984
 626.395194285
 604.5055909949999
 586.0682973535643

In [201]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [202]:
curtailment_costs = [0, 2, 0, 0, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.0
 0.0
 0.0
 0.0

Test without storage first

In [203]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 5
n_eu = 4
# pad year by 1 day
add_h = 24

println("Cost")
C_FC = fixed_cost[1:techs,1]
C_FOM = fixed_cost[1:techs,2]
E_CO2 = co2_intensity[1:techs,1]
C_VOM = variable_cost[1:techs,1]
C_ramp = variable_cost[1:techs,2]
C_start = variable_cost[1:techs,3]
C_curt = curtailment_costs[1:techs]

println("Demand")
D_b = demand[sel_hrs] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h]), D_b, D_b[1:24])
println(size(D_b))
total_demand = D_b
X_D = D_b

A_r = ramp_limits[1:techs, 1]
println("Availability")
A_s = Availability_matrix[sel_hrs,1:techs]
println("Try reverse")
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:techs]), A_s, A_s[1:24, 1:techs])
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
Cost
Demand
(1001,)
(1049,)
Availability
Try reverse
(1049, 5)


In [204]:
println(X_D')

[682.2087979214514 684.3572272808581 683.2104808403138 682.560852210668 677.9666339137987 677.4624380446938 677.183138999593 669.0994848643221 664.1577325922859 660.1332637545196 661.0604433640049 663.0227625822487 662.8027347256789 663.8989970347814 664.9045569967692 675.0512591989451 687.0674878777518 700.1863359290451 707.0921449594324 706.7113499283444 702.3005674276048 688.7704313883322 678.6507224007712 665.7536081253136 751.59574042 766.1490000012872 773.7305163894968 781.5201208059876 788.110369115189 774.0117425390001 751.3358977510001 726.1501180000001 724.689641711 710.6015377680001 667.6450483833117 616.5654627119999 580.4104778550001 557.7014145009999 555.0822907889999 569.547960799 606.7352092409999 638.8923769787382 665.7144059256296 679.2431348193136 686.3113257233966 694.3839132366029 707.2321617094091 716.2252211433974 715.9152202706479 729.1819165424203 749.7397498997365 760.0730590314264 766.4305173375928 764.3226726295725 751.5430568436525 727.7594682174 695.688282

# Constants

In [205]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton
H = 2;

# Model

In [206]:
model = Model(Gurobi.Optimizer) # TODO: options? other optimizers?
set_optimizer_attribute(model, "NonConvex", 2)
#set_optimizer_attribute(model, "IterationLimit", 10000000)
set_optimizer_attribute(model, "TimeLimit", 120)
println("Model: initialized!\n")

n = size(C_FC, 1) # number of technologies
println("Number of technologies: ", n)
#T = size(D_b, 1) # hours in simulation
T = 50
T_start = 987
time_ls_opt = T_start-24:T_start+24

println("Timespan: ", T)
#time_ls = 1+add_h:T-add_h
time_ls = T_start-24+add_h:T_start+24-add_h
vre_ls = 1:2
fossil_ls = 3:4
batt_ind = 5
# Non-negativity constraints
# The installed power capacity of the mix
@variable(model, P[1:n] >= 0)

# The hourly power generation of each technology
@variable(model, X_gen[time_ls_opt, 1:n] >= 0); # hourly production/use for each technology

# Curtailment (only renewables?)
@variable(model, X_cur[time_ls_opt, 1:n] >= 0); # hourly production/use for each technology

###### CONSTRAINTS ######

# Demand constraint - meet demand per hour
println("Demand constraint")
@constraint(model, Con_dh[t = time_ls_opt], sum(X_gen[t, i] for i in 1:n) >= X_D[t]); # TODO: add curtailment
# Meet total demand
# @constraint(model, Con_d, sum(X_gen[t, i] for i in 1:n for t in time_ls_opt) >= sum(total_demand));

# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s);
@constraint(model, Con_wgen, P[2] <= P_max_w);

println("Renewable minimum installed capacity constraint")
#@constraint(model, Con_solar_lb, P[1] >= 50);
#@constraint(model, Con_wind_lb, P[2] >= 600);

# Storage variable
println("Storage variable")
@variable(model, 0<= X_soc[time_ls_opt] <=1)
@variable(model, X_ch[time_ls_opt]>=0)

# Cannot produce more than the installed power
# println("Installed power constraint")
# @constraint(model, Con_p[i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i])

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i]*A_s[t, i])
# Renewables produce maximum possible capacity
@constraint(model, [i = vre_ls, t = time_ls_opt], X_gen[t, i] >= 0.7*P[i]*A_s[t, i]) 


println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i]*A_s[t, i])

println("Curtailment definition")
@constraint(model, [t = time_ls_opt], sum(X_cur[t, i] for i in 1:n) == sum(X_gen[t, i] for i in 1:n) - X_D[t]) 
@constraint(model, [i = 1:n, t = time_ls_opt], X_cur[t, i] <= X_gen[t, i])

# Cannot produce less than the minimum stable generation: X_gen must be zero or min to max
# min_usage = [0, 0.1, 0, 0] #TODO - cant be more than ramp
# @constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] >= P[i]*min_usage[i])

# For each tech, at each hour, change in production cannot exceed ramp rate (TODO might need to convert this to time)
println("Ramp constraint")
# Dummy rate of change variable
# @variable(model, X_roc[time_ls, 1:n])
# @constraint(model, [i = 1:4, t = time_ls], X_roc[t, i] == X_gen[t, i]-X_gen[limit_bound_max(t+1, T), i])

# Auxiliary variables to linearize the rate of change constraint
@constraint(model, Con_rdwn[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) >= -A_r[i]*X_gen[t, i]) 
@constraint(model, Con_rup[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) <= A_r[i]*X_gen[t, i]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 120
Model: initialized!

Number of technologies: 5
Timespan: 50


Demand constraint
Renewable maximum installed capacity constraint
Renewable minimum installed capacity constraint
Storage variable
Availability & installed power constraint
Availability & installed power constraint
Curtailment definition
Ramp constraint


## Objective

In [207]:
# # Add objective function: minimize the total costs TODO: discount rates
# function start_cost(x_0, x_1, i)
#     if x_0 == 0
#         C_start[i]
#     else
#         0
#     end
# end

#TODO RAMP AND STARTUP COSTS

@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
        sum(sum((C_VOM[i] + 100* E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls_opt) +
        sum(sum(C_curt[i] * X_cur[t, i] for i in 1:n) for t in time_ls)
));

# Storage model

Variable to indicate if $x_{g,t}<x_{d,t}$

In [208]:
M = 10000000


# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[time_ls_opt], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in time_ls_opt], sum(X_gen[t,i] for i in 1:4) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in time_ls_opt], -sum(X_gen[t,i] for i in 1:4) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))



1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 963:1011
And data, a 49-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Z_X_gen_greater_X_D_2[963] : -X_gen[963,1] - X_gen[963,2] - X_gen[963,3] - X_gen[963,4] + 10000000 Z_X_gen_greater_X_D[963] ≤ 9.999108660087775e6
 Z_X_gen_greater_X_D_2[964] : -X_gen[964,1] - X_gen[964,2] - X_gen[964,3] - X_gen[964,4] + 10000000 Z_X_gen_greater_X_D[964] ≤ 9.999100179659816e6
 Z_X_gen_greater_X_D_2[965] : -X_gen[965,1] - X_gen[965,2] - X_gen[965,3] - X_gen[965,4] + 10000000 Z_X_gen_greater_X_D[965] ≤ 9.999095473570198e6
 Z_X_gen_greater_X_D_2[966] : -X_gen[966,1] - X_gen[966,2] - X_gen[966,3] - X_gen[966,4] + 10000000 Z_X_gen_greater_X_D[966] ≤ 9.999109131119551

Create the variable X_min = min(X_1, X_2, X_3)

In [209]:
# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[t in time_ls_opt], Bin)
@variable(model, z2[t in time_ls_opt], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in time_ls_opt], eta*P[5] - eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in time_ls_opt], -eta*P[5] + eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[time_ls_opt])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in time_ls_opt], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in time_ls_opt], min_X_1_X_2[t] <= eta*P[5])
@constraint(model, X_1_X_2_higher_1[t in time_ls_opt], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in time_ls_opt], min_X_1_X_2[t] >= eta*P[5] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in time_ls_opt], (1- X_soc[t])* P[5] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in time_ls_opt], - (1- X_soc[t]) * P[5] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[time_ls_opt])

@constraint(model, X_1_X_2_X_3_lower_1[t in time_ls_opt], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in time_ls_opt], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in time_ls_opt], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in time_ls_opt], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{CH}$

In [210]:
#Expression of X_ch
@constraint(model, X_ch_expression[t in time_ls_opt], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [211]:
@variable(model, y2[time_ls_opt], Bin)

# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in time_ls_opt], X_soc[t]* P[5] * H - P[5] <=  M*y2[t])
@constraint(model, y_2_2[t in time_ls_opt], - X_soc[t] * P[5] * H + P[5] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[time_ls_opt])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in time_ls_opt], min_extra_prod_P5_soc[t] <= P[5])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in time_ls_opt], min_extra_prod_P5_soc[t] <= X_soc[t]*P[5]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in time_ls_opt], min_extra_prod_P5_soc[t] >= P[5] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in time_ls_opt], min_extra_prod_P5_soc[t] >= X_soc[t]*P[5]* H - M*y2[t]);



 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{dch}$

In [212]:
@variable(model, batt_dispatch[time_ls_opt], Bin)

#Expression of X_dch
# issue with this constraint
@constraint(model, X_dch_expression[t in time_ls_opt], X_gen[t,5]  <= batt_dispatch[t]*min_extra_prod_P5_soc[t]);



Expression of state of charge

In [213]:
# First hour need to be equal to one
# Start with a charged battery
@constraint(model, SOC_t_1, X_soc[time_ls_opt[1]] == 1)

#
@constraint(model, SOC_t[t in time_ls_opt[2]:time_ls_opt[end]], (X_soc[t]-X_soc[t-1])*H*P[5] == (X_ch[t-1]- X_gen[t-1,5]));


In [214]:
#Optimize
optimize!(model)

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1532 rows, 985 columns and 4738 nonzeros
Model fingerprint: 0x472cf982
Model has 538 quadratic constraints
Variable types: 740 continuous, 245 integer (245 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+07]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+07]
  Objective range  [2e-03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
  QRHS range       [1e+07, 1e+07]
Presolve added 4 rows and 67 columns
Presolve time: 0.01s
Presolved: 2498 rows, 1198 columns, 7177 nonzeros
Presolved model has 239 bilinear constraint(s)
Variable types: 953 continuous, 245 integer (245 binary)

Root relaxation: objective 3.931891e+05, 779 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    

## Results

In [215]:
println(objective_value(model))

2.553299244432971e6


In [216]:
value.(P)

5-element Vector{Float64}:
   97.73788321905455
 2615.9165274987768
  107.0165694657558
    0.0
 1278.5190911581735

In [217]:
X_gen_matrix = value.(X_gen);
    X_gen_matrix = transpose(X_gen_matrix)
size(X_gen_matrix)

(5, 49)

In [218]:
X_cur_matrix = value.(X_cur);
    X_cur_matrix = transpose(X_cur_matrix)
size(X_cur_matrix)

(5, 49)

In [219]:
println(X_D[100-24:100+24])

[744.3539917941578, 748.4950918224149, 744.4231619893369, 735.8894154696181, 716.6670981336667, 703.5208585128647, 679.2019867676861, 672.25726267, 659.3820459949999, 644.380859712, 635.6791651929999, 654.3881033623624, 656.740999109101, 655.8571123, 706.4104289, 735.5282070165002, 737.3771234199531, 742.2084477918578, 751.889204991204, 759.7728924389028, 776.5361659967266, 787.620993084216, 800.7068469324988, 810.3370261866394, 819.8981834945114, 823.971633127109, 816.7667688949764, 788.3429617999999, 756.3938145289999, 761.0252466437886, 747.6733209248807, 704.424120644, 636.594771745, 609.620608668, 603.405887941, 629.436969611, 672.5654317190002, 709.2215659593797, 723.0310515, 751.8306370008507, 755.803804269384, 759.7449524210753, 767.5534577464045, 776.1797991166416, 790.1965149793043, 803.0608772462934, 816.1085327498694, 826.0929902515333, 833.5495396744218]


In [225]:
using Plots
println(T)
@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1, time_ls]
b = X_gen_matrix[2, time_ls]
c = X_gen_matrix[3, time_ls]
d = X_gen_matrix[4, time_ls]
e = X_gen_matrix[5, time_ls]
a_c = X_cur_matrix[1, time_ls] * -1
b_c = X_cur_matrix[2, time_ls] * -1
c_c = X_cur_matrix[3, time_ls] * -1
d_c = X_cur_matrix[4, time_ls] * -1
x = time_ls


time_ls = time_ls_opt
plotly()
stackedarea(x, [d c b a e], labels=["Coal" "CCGT" "Wind" "Solar" "Battery"], lw=0)
stackedarea!(x, [d_c c_c b_c a_c], labels=["Coal, curtail" "CCGT, curtial" "Wind, curtail" "Solar, curtail"], lw=0)
plot!(x, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
# plot!(x, value.(X_D)[:], label="Demand, shifted", lc=:black, lw=2)
plot!(size=(800,400))

50


In [221]:
# Compare availability of solar to wind
plotly()
plot(x, value.(A_s)[time_ls, 1]*P_max_s, label="Solar", lc=:black, lw=1)
plot!(x, a, label="Solar Prod", lc=:blue, lw=1)
plot!(x, value.(A_s)[time_ls, 2]*P_max_w, label="Wind", lc=:black, lw=1, ls=:dash)
plot!(size=(1000,400))

In [222]:
plotly()
ls_X_SOC = []
for elem in value.(X_soc)
    #print(elem)
    ls_X_SOC = [ls_X_SOC; elem]
end
println(ls_X_SOC)


Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9422589430539201, 0.8441660693356879, 0.7385700980013921, 0.6310355665281061, 0.5125502730876984, 0.4058560674764514, 0.30259830402548477, 0.20644136032482746, 0.1304302922050048, 0.06978218168867192, 0.027656547919564043, 0.0, 0.0, 0.07094418952261972, 0.14870944674724967, 0.21996547704969419, 0.28525712549701726, 0.34508363206649767, 0.39990244846124656, 0.4501327359686907, 0.496158570924669, 0.5383318804093984, 0.5769751350024533, 0.6123838123117721, 0.5903847212598937, 0.4978748095783204, 0.39602714820067286, 0.281711167773307, 0.1744526328667449, 0.08313598807662045, 0.022157973988544827, 0.0, 0.0, 0.0, 0.0]


In [223]:
plot(time_ls_opt, ls_X_SOC, label="Battery state of charge", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))

In [224]:
plot(time_ls_opt, value.(X_gen[:,5]), label="X min", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))

LoadError: DenseAxisArray does not support this operation.

# Export the results

In [230]:
using CSV

# Assuming your matrix is named `my_matrix`
my_matrix = [value.(X_gen) value.(X_soc) X_D[time_ls_opt]]  # Replace this with your actual matrix
println(my_matrix)
# Column names

column_names = ["Solar", "Wind", "CCGT", "Coal", "Battery", "State_of_Charge", "Demand"]

# Create a DataFrame with vectors for each column
df = DataFrame([Symbol(name) => my_matrix[:, i] for (i, name) in enumerate(column_names)])

# Specify the file path for your CSV file
csv_file_path = "Results/output.csv"

# Export the DataFrame to CSV
CSV.write(csv_file_path, df)

println("CSV file exported to: $csv_file_path")


[47.12033695957731 844.2195752641919 0.0 0.0 0.0 1.0 891.3399122237693; 70.56121698330647 829.2591232008581 0.0 0.0 0.0 1.0 899.8203401841646; 69.491679146271 971.0207452579017 0.0 0.0 0.0 1.0 904.5264298028051; 44.92953269958828 1055.7884799814963 0.0 0.0 0.0 1.0 890.868880448; 55.03953509794316 1056.5669678459637 0.0 0.0 0.0 1.0 855.272268728; 42.77894444115785 1050.8093496948884 0.0 0.0 0.0 1.0 821.4399951570001; 19.9870321748145 1009.8482454562785 0.0 0.0 0.0 1.0 813.6654385040001; 10.43366722381261 968.81910149038 0.0 0.0 0.0 1.0 806.8007585649524; 4.3876001649806105 956.7251292329846 0.0 0.0 0.0 1.0 760.5865213710001; 1.1692410336102799 1322.2632777160222 0.0 0.0 0.0 1.0 685.1299108050001; 0.08534208070403151 1277.5786258354315 0.0 0.0 0.0 1.0 658.844224492; 0.0 862.557608309954 0.0 0.0 0.0 1.0 652.2367688319999; 0.0 674.6131123952185 0.0 0.0 0.0 1.0 674.6131123959999; 0.0 564.3549096638249 0.0 0.0 147.64608722217554 1.0 712.000996886; 0.0 507.83393917784826 0.0 0.0 250.827223442

In [229]:
for name in column_names
    println("output_"* name* ".csv")
end

outputSolar.csv
outputWind.csv
outputCCGT.csv
outputCoal.csv
outputBattery.csv
outputState_of_Charge.csv
outputDemand.csv


In [237]:
name_to_store = "test"
println("Test")
# Assuming your matrix is named `my_matrix`
Power = value.(P)   # Replace this with your actual matrix
# Column names
println(Power)
column_names_power = "Installed Power"

# Create a DataFrame with vectors for each column
df_Power = DataFrame(Symbol(column_names_power) => Power)

# Specify the file path for your CSV file
csv_file_path_power = "Results/output_Power" * name_to_store* ".csv"

# Export the DataFrame to CSV
CSV.write(csv_file_path_power, df_Power)

println("CSV file exported to: $csv_file_path_power")

Test
[97.73788321905455, 2615.9165274987768, 107.0165694657558, 0.0, 1278.5190911581735]
CSV file exported to: Results/output_Powertest.csv
